In [1]:
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np 
import pandas as pd
import tensorflow as tf
import re
from collections import  Counter
from sklearn.model_selection import train_test_split 
from torchmetrics import R2Score
from torch.nn import MSELoss
from torch.optim import Adam
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
rating=pd.read_csv(r'D:\movrec\data_preprocessing\cpdata4v2\ratings.csv')
movies=pd.read_csv(r'D:\movrec\data_preprocessing\cpdata3v2\movies.csv')
user_num=len(pd.unique(rating['userId']))+1
movie_num=len(pd.unique(rating['movieId']))+1
print(user_num,movie_num)
train,test=train_test_split(rating,test_size=0.09,stratify=rating['userId'])
train=train.to_numpy()
test=test.to_numpy()
movies.columns


151968 9448


Index(['Movie ID', 'Title', 'Overview', 'Genres', 'Release Date', 'Runtime',
       'Average Rating', 'Vote Count', 'Popularity', 'Budget', 'Adult',
       'Poster Path', 'Backdrop Path', 'Cast', 'Director', 'Keywords',
       'Production Companies', 'Production Countries', 'Similar Movies',
       'Movie Homepage'],
      dtype='object')

In [3]:
user_train=torch.tensor(train[:,0],dtype=torch.long)
movie_train=torch.tensor(train[:,1],dtype=torch.int32)
rating_train=torch.tensor(train[:,2],dtype=torch.float32)
training_data=torch.utils.data.DataLoader(torch.utils.data.TensorDataset(user_train,movie_train,rating_train),batch_size=128)
#delete unneed data
user_train=None
movie_train=None
rating_train=None
user_val=torch.tensor(test[:,0],dtype=torch.long)
movie_val=torch.tensor(test[:,1],dtype=torch.int32)
rating_val=torch.tensor(test[:,2],dtype=torch.float32)
dev_data=torch.utils.data.DataLoader(torch.utils.data.TensorDataset(user_val,movie_val,rating_val),batch_size=1024)
#delete unneed data
user_val=None
movie_val=None
rating_val=None
#sort by movie id to ensure bring data with correct index because  the range of data is from 1 to 7616 with no trap
movies.sort_values(by='Movie ID',inplace=True)
movies.drop(labels=['Movie ID','Poster Path','Backdrop Path','Keywords','Similar Movies','Movie Homepage'],axis=1,inplace=True)
movies['Release Date']=pd.to_datetime(movies['Release Date'])
numeric_movie_data=movies.select_dtypes(include=['number','bool','datetime'])
movies.drop(labels=numeric_movie_data.columns,axis=1,inplace=True)
numeric_movie_data['year']=numeric_movie_data['Release Date'].dt.year
numeric_movie_data['month']=numeric_movie_data['Release Date'].dt.month
numeric_movie_data['day']=numeric_movie_data['Release Date'].dt.day
numeric_movie_data.drop(labels=['Release Date','Average Rating','Vote Count'],axis=1,inplace=True)
print(numeric_movie_data.columns)
numeric_movie_data=torch.tensor(numeric_movie_data.to_numpy().astype(np.float32),device=device,dtype=torch.float32)
max_movie_id = int(torch.max(torch.tensor(train[:,1], dtype=torch.long))) - 1
if max_movie_id >= numeric_movie_data.shape[0]:
    raise ValueError(f"Movie id out of bounds: {max_movie_id} vs. numeric_movie_data rows {numeric_movie_data.shape[0]}")

train=None
test=None
rating=None


Index(['Runtime', 'Popularity', 'Budget', 'Adult', 'year', 'month', 'day'], dtype='object')


In [4]:
numeric_movie_data.shape

torch.Size([9447, 7])

In [5]:
for i in range(numeric_movie_data.shape[1]):
    avg=torch.mean(numeric_movie_data[:,i])
    std=torch.std(numeric_movie_data[:,i])
    if std==torch.nan:
        std=1
    if i!=3:
        numeric_movie_data[:,i]=(numeric_movie_data[:,i]-avg)/std


In [6]:
numeric_movie_data[:5,:]

tensor([[-1.5418, -0.5043, -0.5865,  0.0000, -0.5148,  0.8822,  0.6429],
        [-1.4963, -0.5135, -0.5865,  0.0000, -0.6210,  0.8822,  0.1742],
        [-0.4032, -0.3931, -0.4832,  0.0000, -0.1429,  1.4700, -0.7632],
        [ 0.0978, -0.4840, -0.0445,  0.0000, -0.2492,  0.8822, -0.0602],
        [ 0.6443,  0.6326, -0.3026,  0.0000, -1.0992, -0.5873,  1.1116]],
       device='cuda:0')

In [7]:
class Tokenizer:
    def __init__(self,split,max_seq_len):
        self.tokn=list()
        self.vocab=list()
        self.index=dict()
        self.num=list()
        self.max_len=max_seq_len
        self.split=split
    def standerization(self,data:str,split:str):
        data=str(data)
        data=data.lower()
        data=re.sub('[\.@#%^&*()!:;\\\$\'"]*',"",data)
        if split != None:
            data=re.split(split,data)
            data=list(map(lambda x:re.sub(' ',"",x),data))
        else:
            data=re.sub(r' ',"",data)
        return data
    def add_to_vocab(self,row):
        if isinstance(row, list):
            for word in row:
                if word not in self.vocab:
                    self.vocab.append(word)
        elif isinstance(row,str):
            if row not in self.vocab:
                self.vocab.append(row)
    def build_vocab(self):
        for li in self.tokn: 
            self.add_to_vocab(li)
    def maper(self):

        self.index={word : idx+2 for idx,word in enumerate(self.vocab)}
        for idx,li in enumerate(self.tokn):
            if isinstance(li, list):
                self.num.append(list())
                for word in li:
                    self.num[idx].append(self.index[word])
            else:
                self.num.append(self.index[li])
    def padding(self):

        for i in self.num:
            if not isinstance(i, list):
                break
            while len(i)>self.max_len:
                i.pop(-1)
            while len(i)<self.max_len:
                i.append(1)
    def transform(self,data):
        self.tokn=list(map(lambda word:self.standerization(word,self.split),data))
        c=0
        su=0
        if self.split!=None:
            for i in self.tokn:
                su+=len(i)
                c+=1
            print(su/c)
        self.build_vocab()
        self.maper()
        self.padding()
        return  self.num
title=torch.tensor(Tokenizer(max_seq_len=1,split=None).transform(movies['Title']),device=device,dtype=torch.long)
cast=torch.tensor(Tokenizer(max_seq_len=3,split=',').transform(movies['Cast'].to_list()),device=device,dtype=torch.long)
genre=torch.tensor(Tokenizer(max_seq_len=4,split=',').transform(movies['Genres'].to_list()),device=device,dtype=torch.long)
overrview=torch.tensor(Tokenizer(max_seq_len=50,split=' ').transform(movies['Overview'].to_list()),device=device,dtype=torch.long)
director=torch.tensor(Tokenizer(max_seq_len=1,split=',').transform(movies['Director'].to_list()),device=device,dtype=torch.long).squeeze()
production_compaines=torch.tensor(Tokenizer(max_seq_len=3,split=',').transform(movies['Production Companies'].to_list()),device=device,dtype=torch.long)
production_countries=torch.tensor(Tokenizer(max_seq_len=2,split=',').transform(movies['Production Countries'].to_list()),device=device,dtype=torch.long)
movies=None


4.9787234042553195
2.6343812850640416
46.69344765534032
1.0370487985603896
3.2065205885466286
1.4553826611622738


In [8]:
class FunctionalModel(torch.nn.Module):
    def __init__(self, user_size):
        super(FunctionalModel, self).__init__()
        self.user_emb = torch.nn.Embedding(user_size, 100)
        self.title_emb=torch.nn.Embedding(torch.max(title)+1,20)
        self.overreview_emb=torch.nn.Embedding(torch.max(overrview)+1,20)
        self.director_emb=torch.nn.Embedding(torch.max(director)+1,8)
        self.cast_emb=torch.nn.Embedding(torch.max(cast)+1,10)
        self.genre_emb=torch.nn.Embedding(torch.max(genre)+1,15)
        self.prod_comp_emb=torch.nn.Embedding(torch.max(production_compaines)+1,10)
        self.prod_count_emb=torch.nn.Embedding(torch.max(production_countries)+1,10)
        self.dropout=torch.nn.Dropout(0.2)
    def forward(self, user_ids, movie_ids):
        #because index of tokenized data start from 0 not one 
        movie_ids=movie_ids-1 
        user =self.user_emb(user_ids)
        tit=self.title_emb(title[movie_ids])
        ovrv=self.overreview_emb(overrview[movie_ids])
        dire=self.director_emb(director[movie_ids])
        ct=self.cast_emb(cast[movie_ids])
        gn=self.genre_emb(genre[movie_ids])
        pd_cmp=self.prod_comp_emb(production_compaines[movie_ids])
        pd_count=self.prod_count_emb(production_countries[movie_ids])
        num_data=numeric_movie_data[movie_ids]
        #because the diff of sequence  we must reduce the dim without lossing of data
        ovrv_vec=ovrv.mean(dim=1)
        ct_vec=ct.mean(dim=1)
        gn_vec=gn.mean(dim=1)
        pd_cmp_vec=pd_cmp.mean(dim=1)
        pd_count_vec=pd_count.mean(dim=1) 
        movie=torch.cat((tit,ovrv_vec,dire,ct_vec,gn_vec,pd_cmp_vec,pd_count_vec,num_data),dim=-1)
        movie=self.dropout(movie)
        user=self.dropout(user)
        x = torch.sum(user * movie, dim=-1, keepdim=True)
        return x.squeeze()

In [11]:
hybrid_base_model=FunctionalModel(user_num)
path=r'D:\movrec\model\hybrid model\hybird_base_model\base_model_MAE_0.7609.pth'
hybrid_base_model.load_state_dict(torch.load(path))
hybrid_base_model=hybrid_base_model.to(device)

In [12]:
adam=Adam(hybrid_base_model.parameters(),lr=1e-4)
metric=R2Score().to(device)
critery=torch.nn.L1Loss()
torch.backends.cudnn.benchmark = True
epoch=5
for ep in range(epoch):
    hybrid_base_model.train()
    metric.reset()
    runing_loss=0.0
    for idx,data in enumerate(training_data):
        u,m,r=data
        adam.zero_grad()
        u=u.to(device)
        m=m.to(device)
        r=r.to(device)
        output=hybrid_base_model(u,m)        
        loss=critery(output,r)
        loss.backward()
        adam.step()
        runing_loss+=loss.item()
        metric.update(output.squeeze(),r.squeeze())
        str_out=f'{idx}\{len(training_data)}, loss is {runing_loss/(idx+1):.4}\r'
        print(str_out,end="")
    result=metric.compute()
    print(len(str_out)*"",end='')
    print(f'{ep+1}\{epoch}, loss is {runing_loss/len(training_data):.4}, R2 result is {result:.4}', end="")
    hybrid_base_model.eval()
    metric.reset()
    runing_loss=0.0
    with torch.no_grad():
        for u,m,r in dev_data:
            u=u.to(device)
            m=m.to(device)
            r=r.to(device)
            output=hybrid_base_model(u,m)
            loss=critery(output.squeeze(),r.squeeze())
            runing_loss+=loss.item()
            metric.update(output,r)
        result=metric.compute()
        print(f',loss of dev{runing_loss/len(dev_data)}, with R2 {result}')
    
    

1\5, loss is 0.704, R2 result is 0.2304,loss of dev0.5782549159024435, with R2 0.45673292875289917
2\5, loss is 0.6775, R2 result is 0.2828,loss of dev0.5718418915470983, with R2 0.4655442237854004
3\5, loss is 0.6686, R2 result is 0.3001,loss of dev0.5699130545195182, with R2 0.46868211030960083
4\5, loss is 0.6636, R2 result is 0.3101,loss of dev0.5699675238803773, with R2 0.4697972536087036
5\5, loss is 0.6602, R2 result is 0.3165,loss of dev0.5683957577617899, with R2 0.4702790379524231


In [13]:
path=r'D:\movrec\model\hybrid model\hybird_base_model'
import os
os.makedirs(path,exist_ok=True)
torch.save(hybrid_base_model.state_dict(),path+'\\base_model_MAE_0.6602.pth')

In [14]:
path=r'D:\movrec\model\toknized_tensor'
torch.save(title,path+"\\title.pt")
torch.save(overrview,path+"\\overrview.pt")
torch.save(director,path+"\\director.pt")
torch.save(cast,path+"\\cast.pt")
torch.save(production_compaines,path+'\\production_compaines.pt')
torch.save(production_countries,path+'\\production_countries.pt')
torch.save(genre,path+'\\genre.pt')
torch.save(numeric_movie_data,path+'\\numeric_movie_data.pt')